In [149]:
import csv
import os
from io import StringIO

import time
from calendar import monthrange

from tqdm import tqdm
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [150]:
caiso = 'http://www.caiso.com/outlook/SP/History/'

YEARS = [2020, 2021]
MONTHS = [f"0{x}" for x in range(1, 10)] + ["10", "11", "12"]

# Calculate Day Strings for Query
day_strings = []
for year in YEARS:
    for month in MONTHS:
        num_days = monthrange(year, int(month))[1]
        day_strings += [f"{year}{month}0{d}" for d in range(1, 10)] + [f"{year}{month}{d}" for d in range(10, num_days + 1)]

In [151]:
def get_suppy_data(day_num):
    """ Makes a request to Caiso and returns CSV formatted data """
    r = requests.get(f"{caiso}{day_num}/fuelsource.csv?_=1642727187499")
    soup = bs(r.text, 'html.parser')
    csv = StringIO(str(soup))
    return csv

In [152]:
""" Create Empty DataFrame, fill up by grabbing data """
all_df = pd.DataFrame()

for day in tqdm(day_strings):
    data = get_suppy_data(day)
    new_df = pd.read_csv(data, sep=",")
    new_df['Day'] = [day for r in range(len(new_df['Time']))]
    all_df = pd.concat([all_df, new_df])
    time.sleep(0.1)  # Being kind to the Caiso Site

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 731/731 [04:50<00:00,  2.51it/s]


In [153]:
""" Formatting, name correction """ 
all_df.index = list(range(len(all_df['Time'])))
all_df['Natural Gas'] = all_df['Natural Gas'].combine_first(all_df['Natural gas'])
all_df['Large Hydro'] = all_df['Large Hydro'].combine_first(all_df['Large hydro'])
all_df = all_df.drop(columns=['Natural gas', 'Large hydro'])
all_df.head()

,Time,Solar,Wind,Geothermal,Biomass,Biogas,Small hydro,Coal,Nuclear,Batteries,Imports,Other,Day,Natural Gas,Large Hydro
0,0:00,-33.0,808.0,651.0,305.0,223.0,222.0,11.0,2274.0,0.0,8653.0,0.0,20200101,7871.0,1561.0
1,0:05,-33.0,804.0,651.0,305.0,227.0,214.0,11.0,2273.0,9.0,8546.0,0.0,20200101,8143.0,1438.0
2,0:10,-33.0,800.0,651.0,283.0,229.0,205.0,11.0,2274.0,0.0,8395.0,0.0,20200101,8437.0,1348.0
3,0:15,-33.0,790.0,650.0,305.0,230.0,206.0,10.0,2273.0,7.0,8350.0,0.0,20200101,8453.0,1317.0
4,0:20,-33.0,790.0,649.0,305.0,230.0,205.0,10.0,2273.0,4.0,8339.0,0.0,20200101,8402.0,1339.0


In [154]:
""" Save to CSV """
all_df.to_csv("california_genmix_20200101_20211231.csv")

In [156]:
all_df

,Time,Solar,Wind,Geothermal,Biomass,Biogas,Small hydro,Coal,Nuclear,Batteries,Imports,Other,Day,Natural Gas,Large Hydro
0,0:00,-33.0,808.0,651.0,305.0,223.0,222.0,11.0,2274.0,0.0,8653.0,0.0,20200101,7871.0,1561.0
1,0:05,-33.0,804.0,651.0,305.0,227.0,214.0,11.0,2273.0,9.0,8546.0,0.0,20200101,8143.0,1438.0
2,0:10,-33.0,800.0,651.0,283.0,229.0,205.0,11.0,2274.0,0.0,8395.0,0.0,20200101,8437.0,1348.0
3,0:15,-33.0,790.0,650.0,305.0,230.0,206.0,10.0,2273.0,7.0,8350.0,0.0,20200101,8453.0,1317.0
4,0:20,-33.0,790.0,649.0,305.0,230.0,205.0,10.0,2273.0,4.0,8339.0,0.0,20200101,8402.0,1339.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210523,23:35,-33.0,3778.0,882.0,287.0,203.0,187.0,20.0,2264.0,-144.0,8331.0,0.0,20211231,7287.0,1205.0
210524,23:40,-32.0,3755.0,882.0,285.0,203.0,188.0,20.0,2265.0,-176.0,8414.0,0.0,20211231,7245.0,1206.0
210525,23:45,-31.0,3751.0,882.0,285.0,205.0,188.0,20.0,2265.0,-234.0,8505.0,0.0,20211231,7222.0,1216.0
210526,23:50,-32.0,3793.0,882.0,284.0,203.0,188.0,20.0,2264.0,-294.0,8526.0,0.0,20211231,7239.0,1208.0
